# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/city_data.csv')
cities_df.dropna(inplace = True) 
cities_df.head(20)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Viedma,-40.8135,-62.9967,69.80,26,9,18.41,AR,1628099170
1,Bluff,-46.6000,168.3333,35.80,76,100,3.47,NZ,1628099171
2,Hobart,-42.8794,147.3294,48.06,70,40,3.00,AU,1628099016
3,Tucumã,-0.1833,-50.0500,84.45,64,23,4.88,BR,1628099025
4,Yellowknife,62.4560,-114.3525,54.32,54,90,5.75,CA,1628099171
5,Ambanja,-13.6833,48.4500,72.03,86,49,2.26,MG,1628099172
6,Clyde River,70.4692,-68.5914,37.63,93,90,9.22,CA,1628098872
8,Skjervøy,70.0311,20.9714,53.60,95,100,8.43,NO,1628099173
9,Mogadishu,2.0371,45.3438,76.08,86,75,17.54,SO,1628099173
10,Tasiilaq,65.6145,-37.6368,53.76,75,96,0.76,GL,1628099173


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [69]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center=(35.0, -30.0), zoom_level=2.4)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [145]:
# Good conditions for golf
#
# A max temperature lower than 79 degrees but higher than 69.
# Wind speed less than 5 mph
# Humidity less than 40

cities_df.head(20)
hotel_df = cities_df[(cities_df['Max Temp'] < 89) & 
                        (cities_df['Max Temp'] > 69) & 
                        (cities_df['Wind Speed'] < 7) & 
                        (cities_df['Humidity'] < 50)
                       ]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,High Prairie,55.4334,-116.4858,77.79,37,30,5.75,CA,1628099174
68,Saint Anthony,45.0205,-93.2180,85.78,44,1,5.01,US,1628099196
79,Athabasca,54.7169,-113.2854,73.22,29,92,3.80,CA,1628099200
125,San Vicente,-26.6167,-54.1333,74.03,23,19,2.51,AR,1628099218
134,Manono,-7.3000,27.4167,82.15,29,0,2.08,CD,1628099222
137,Korla,41.7597,86.1469,85.57,20,100,4.52,CN,1628098948
169,Kananga,-5.8958,22.4178,81.14,24,3,4.54,CD,1628099234
182,Stonewall,50.1344,-97.3268,77.67,30,100,3.31,CA,1628099239
202,Meadow Lake,34.8014,-106.5436,83.48,42,1,5.06,US,1628099114
210,Klamath Falls,42.2249,-121.7817,85.87,35,20,0.00,US,1628099063


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [146]:
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-146-cb2d5672db59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,High Prairie,55.4334,-116.4858,77.79,37,30,5.75,CA,1628099174,
68,Saint Anthony,45.0205,-93.2180,85.78,44,1,5.01,US,1628099196,
79,Athabasca,54.7169,-113.2854,73.22,29,92,3.80,CA,1628099200,
125,San Vicente,-26.6167,-54.1333,74.03,23,19,2.51,AR,1628099218,
134,Manono,-7.3000,27.4167,82.15,29,0,2.08,CD,1628099222,
137,Korla,41.7597,86.1469,85.57,20,100,4.52,CN,1628098948,
169,Kananga,-5.8958,22.4178,81.14,24,3,4.54,CD,1628099234,
182,Stonewall,50.1344,-97.3268,77.67,30,100,3.31,CA,1628099239,
202,Meadow Lake,34.8014,-106.5436,83.48,42,1,5.06,US,1628099114,
210,Klamath Falls,42.2249,-121.7817,85.87,35,20,0.00,US,1628099063,


In [147]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

C:\Users\Mastermind\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,High Prairie,55.4334,-116.4858,77.79,37,30,5.75,CA,1628099174,Peavine Inn & Suites
68,Saint Anthony,45.0205,-93.2180,85.78,44,1,5.01,US,1628099196,Hewing Hotel
79,Athabasca,54.7169,-113.2854,73.22,29,92,3.80,CA,1628099200,Days Inn by Wyndham Athabasca
125,San Vicente,-26.6167,-54.1333,74.03,23,19,2.51,AR,1628099218,Hotel Y Restaurante Los Lapachos
134,Manono,-7.3000,27.4167,82.15,29,0,2.08,CD,1628099222,Hotel hewa bora
137,Korla,41.7597,86.1469,85.57,20,100,4.52,CN,1628098948,The Garden Hotel Korla
169,Kananga,-5.8958,22.4178,81.14,24,3,4.54,CD,1628099234,Grand Hotel De Kananga
182,Stonewall,50.1344,-97.3268,77.67,30,100,3.31,CA,1628099239,Rockwood Motor Inn & Motel
202,Meadow Lake,34.8014,-106.5436,83.48,42,1,5.06,US,1628099114,
210,Klamath Falls,42.2249,-121.7817,85.87,35,20,0.00,US,1628099063,Fairfield Inn & Suites by Marriott Klamath Falls


In [151]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [152]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Peavine Inn & Suites</dd>\n<dt>City</dt><dd>High Prairie</dd>\n<dt>Country</dt><dd>CA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hewing Hotel</dd>\n<dt>City</dt><dd>Saint Anthony</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Days Inn by Wyndham Athabasca</dd>\n<dt>City</dt><dd>Athabasca</dd>\n<dt>Country</dt><dd>CA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Y Restaurante Los Lapachos</dd>\n<dt>City</dt><dd>San Vicente</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel hewa bora</dd>\n<dt>City</dt><dd>Manono</dd>\n<dt>Country</dt><dd>CD</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>The Garden Hotel Korla</dd>\n<dt>City</dt><dd>Korla</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Grand Hotel De Kananga</dd>\n<dt>City</dt><dd>Kananga</dd>\n<dt>Country</dt><dd>CD</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Rockwood Motor Inn & Motel</dd>\n<dt>City</dt><dd>Stonewall</dd>\n<dt>Country</

In [156]:
hotel_layer = gmaps.symbol_layer(
    locations, 
    fill_color='red',
    stroke_color='rgba(0, 0, 150, 0.4)', 
    scale=6,
    info_box_content=[f"{hotel}" for hotel in hotel_info]
)


fig = gmaps.figure(center=(30.0, -30.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))